In [1]:
%load_ext sql

In [7]:
import requests
import os
import pandas as pd
from sqlalchemy import create_engine
from faker import Faker

In [3]:
pd.set_option('display.max_rows', 12)
pd.set_option('display.min_rows', 6)

In [4]:
fake = Faker()

base_url = "http://fastapi:8000"
headers = {"Content-Type": "application/json"}

print("customers")
# Create customers
for customer_id in range(1, 11):
    data = {
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "email": fake.email(),
        "joined_at": fake.iso8601()
    }
    response = requests.post(f"{base_url}/customers/", json=data, headers=headers)
    print(response.status_code, response.json())

print("vendors")
# Create vendors
for vendor_id in range(1, 6):
    data = {
        "vendor_name": fake.company(),
        "region": fake.country_code()
    }
    response = requests.post(f"{base_url}/vendors/", json=data, headers=headers)
    print(response.status_code, response.json())

print("inventory per vendor")
# Add inventory per vendor
for vendor_id in range(1, 6):
    for item_id in range(1, 11):
        params = {"vendor_id": vendor_id}
        data = {
            "item_name": fake.word(),
            "category": fake.word(),
            "price": fake.random_int(min=10, max=500),
        }
        response = requests.post(f"{base_url}/inventory/", json=data, headers=headers, params=params)
        print(response.status_code, response.json())

print("cart and checkout twice and add to cart again!")
for customer_id in range(1, 11):
    for _ in range(2):  # Checkout twice
        cart_data = {
            "user_id": customer_id,
            "cart": [
                {
                    "item_id": str(fake.random_int(min=1, max=10)),
                    "vendor_id": str(fake.random_int(min=1, max=5)),
                    "qty": fake.random_int(min=1, max=5)
                }
            ]
        }
        response = requests.post(f"{base_url}/cart/", json=cart_data, headers=headers)
        print(response.status_code, response.json())
        
        response = requests.post(f"{base_url}/checkout/", json={"user_id": customer_id}, headers=headers)
        print(response.status_code, response.text)
    
    # Add to cart one more time
    cart_data = {
        "user_id": customer_id,
        "cart": [
            {
                "item_id": str(fake.random_int(min=1, max=10)),
                "vendor_id": str(fake.random_int(min=1, max=5)),
                "qty": fake.random_int(min=1, max=5)
            }
        ]
    }
    response = requests.post(f"{base_url}/cart/", json=cart_data, headers=headers)
    print(response.status_code, response.json())

customers
201 {'id': 1, 'first_name': 'Christine', 'last_name': 'Taylor', 'email': 'lewisrachel@example.com', 'joined_at': '2025-03-16T10:50:51.811274'}
201 {'id': 2, 'first_name': 'Ariel', 'last_name': 'Vance', 'email': 'susancruz@example.org', 'joined_at': '2025-03-16T10:50:51.822178'}
201 {'id': 3, 'first_name': 'Tony', 'last_name': 'Williams', 'email': 'justin87@example.net', 'joined_at': '2025-03-16T10:50:51.832528'}
201 {'id': 4, 'first_name': 'Jack', 'last_name': 'Klein', 'email': 'evanscaroline@example.net', 'joined_at': '2025-03-16T10:50:51.841688'}
201 {'id': 5, 'first_name': 'Corey', 'last_name': 'Taylor', 'email': 'audrey99@example.net', 'joined_at': '2025-03-16T10:50:51.850907'}
201 {'id': 6, 'first_name': 'Jason', 'last_name': 'Horton', 'email': 'lance09@example.org', 'joined_at': '2025-03-16T10:50:51.860058'}
201 {'id': 7, 'first_name': 'Jose', 'last_name': 'Sanchez', 'email': 'amanda83@example.org', 'joined_at': '2025-03-16T10:50:51.868223'}
201 {'id': 8, 'first_name': 

In [8]:
OLTP_USER = os.getenv("POSTGRES_USER")
OLTP_PASS = os.getenv("POSTGRES_PASSWORD")
OLTP_HOST = os.getenv("POSTGRES_DB")
OLTP_DB_NAME = os.getenv("POSTGRES_HOST")
OLAP_USER = os.getenv("REDSHIFT_USER")
OLAP_PASS = os.getenv("REDSHIFT_PASSWORD")
OLAP_HOST = os.getenv("REDSHIFT_DB")
OLAP_DB_NAME = os.getenv("REDSHIFT_HOST")

In [9]:
connection_string = f"postgresql://{OLTP_USER}:{OLTP_PASS}@db:5432/{OLTP_DB_NAME}"
engine = create_engine(connection_string)
get_ipython().run_line_magic('sql', connection_string)

RuntimeError: (psycopg2.OperationalError) connection to server at "db" (172.18.0.3), port 5432 failed: FATAL:  password authentication failed for user "None"

(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
%sql \dt

In [ ]:
table_list = [
    "customer", "inventory", "vendor"
]

In [ ]:
for table in table_list:
    print(f"{table}")
    schema_query = f"columns --table {table}"
    display(get_ipython().run_line_magic('sqlcmd', schema_query))

In [ ]:
for table in table_list:
    print(f"{table}")
    display(pd.read_sql(f"select * from {table} limit 100", engine))